In [ ]:
import pandas as pd
import numpy as np

# Carregar CSV
df = pd.read_csv("CSV_Jogo10_VF.xls", delimiter=";")

# Corrigir vírgulas em números e converter colunas numéricas
df["Velocidade maxima [km/h]"] = df["Velocidade maxima [km/h]"].str.replace(',', '.').astype(float)

# Converter colunas de tempo para segundos
def tempo_para_minutos(t):
    h, m, s = [int(x) for x in t.split(":")]
    return h * 60 + m + s / 60

df["Minutos Jogo"] = df["Duracao"].apply(tempo_para_minutos)

### FC Z4 e Z5 para intensidade cardíaca

In [ ]:
df["Min FC Z4"] = df["Tempo na zona FC 4 (80 - 89 %)"].apply(tempo_para_minutos)
df["Min FC Z5"] = df["Tempo na zona FC 5 (90 - 100 %)"].apply(tempo_para_minutos)
df["Min FC Alta"] = df["Min FC Z4"] + df["Min FC Z5"]

### Distâncias em zonas de velocidade

In [ ]:
df["Z5 [m]"] = df["Distancia na zona de velocidade 5 [m] (19.8 - 25 km/h)"].astype(float)
df["Z6 [m]"] = df["Distancia na zona de velocidade 6 [m] (> 25.2 km/h)"].astype(float)
df["Alta Intensidade [m]"] = df["Z5 [m]"] + df["Z6 [m]"]

### Acelerações


In [ ]:
df["Acel Z2"] = df["Numero de aceleracoes z2 (1.00 - 1.99 m/s²)"].astype(int)
df["Acel Z3"] = df["Numero de aceleracoes z3 (2.00 - 2.99 m/s²)"].astype(int)
df["Total Acel"] = df["Acel Z2"] + df["Acel Z3"]

### Desacelerações

In [ ]:
df["Desacel Z2"] = df["Numero de desaceleracoes z2 (-2.99 - -2.00 m/s²)"].astype(int)
df["Desacel Z3"] = df["Numero de desaceleracoes z3 (-1.99 - -1.00 m/s²)"].astype(int)
df["Total Desacel"] = df["Desacel Z2"] + df["Desacel Z3"]

### Distância relativa

In [ ]:
df["Dist Total [m]"] = df["Distancia total [m]"].astype(float)
df["Dist Relativa (m/min)"] = df["Dist Total [m]"] / df["Minutos Jogo"]
df["Alta Intensidade Relativa (m/min)"] = df["Alta Intensidade [m]"] / df["Minutos Jogo"]

## Resultado final com os KPI’s principais

In [ ]:
kpis = df[[
    "Jogador", "Posicao do jogador", "Minutos Jogo", "Dist Total [m]",
    "Z5 [m]", "Z6 [m]", "Alta Intensidade [m]",
    "Numero Sprints", "Acel Z2", "Acel Z3", "Total Acel",
    "Desacel Z2", "Desacel Z3", "Total Desacel",
    "Velocidade maxima [km/h]", "Dist Relativa (m/min)",
    "Alta Intensidade Relativa (m/min)",
    "Min FC Z4", "Min FC Z5", "Min FC Alta"
]]

kpis.head(10)

In [ ]:
# Filtrar apenas as colunas numéricas
kpis_numericas = kpis.select_dtypes(include=[np.number])

# Agrupar por posição com média e desvio padrão
agrupado = kpis.groupby("Posicao do jogador")[kpis_numericas.columns].agg(["mean", "std"])

# Filtrar jogadroes com mais de 60 minutos
kpis[kpis["Minutos Jogo"] > 60]

# Exportar para Excel
kpis.to_excel("Relatorio_Jogo10.xlsx", index=False)

In [ ]:
# Junta média ± desvio como string formatada (ex: 10123 ± 450)
tabela_formatada = agrupado.copy()

for col in kpis_numericas.columns:
    media = agrupado[(col, 'mean')]
    desvio = agrupado[(col, 'std')]
    tabela_formatada[col] = media.round(1).astype(str) + " ± " + desvio.round(1).astype(str)

# Apenas manter as colunas formatadas (removendo multi-index)
tabela_formatada = tabela_formatada[kpis_numericas.columns]

# Visualizar as primeiras linhas
tabela_formatada.head()

In [ ]:
jogadores_60min.shape



In [ ]:
# Filtrar jogadores com mais de 60 minutos
jogadores_45min = kpis[kpis["Minutos Jogo"] >= 45]

# Criar dicionário para armazenar os nomes dos jogadores
jogadores_maximos = {}
jogadores_minimos = {}

# Selecionar colunas numéricas
colunas_numericas = jogadores_45min.select_dtypes(include=[np.number]).columns

# Para cada KPI, encontrar o nome do jogador com o valor máximo e mínimo
for col in colunas_numericas:
    max_idx = jogadores_45min[col].idxmax()
    min_idx = jogadores_45min[col].idxmin()
    
    jogador_max = jogadores_45min.loc[max_idx, "Jogador"]
    jogador_min = jogadores_45min.loc[min_idx, "Jogador"]
    
    jogadores_maximos[col] = jogador_max
    jogadores_minimos[col] = jogador_min

# Criar tabela final
tabela_extremos_com_nomes = pd.DataFrame({
    "KPI": colunas_numericas,
    "Máximo": jogadores_45min[colunas_numericas].max().values,
    "Jogador (máximo)": [jogadores_maximos[col] for col in colunas_numericas],
    "Mínimo": jogadores_45min[colunas_numericas].min().values,
    "Jogador (mínimo)": [jogadores_minimos[col] for col in colunas_numericas],
})

# Ver a tabela
tabela_extremos_com_nomes



## Gráficos

### Gráfico 1: Distância total por jogador

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.barplot(x="Jogador", y="Dist Total [m]", data=kpis, palette="Blues_d")
plt.xticks(rotation=90)
plt.title("Distância Total Percorrida por Jogador")
plt.ylabel("Distância (m)")
plt.tight_layout()
plt.savefig("grafico_distancia_total.png")

## Gráfico 2: Distância em alta intensidade

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x="Jogador", y="Alta Intensidade [m]", data=kpis, palette="Oranges_d")
plt.xticks(rotation=90)
plt.title("Distância em Corrida de Alta Intensidade (Z5 + Z6)")
plt.ylabel("Distância (m)")
plt.tight_layout()
plt.savefig("grafico_alta_intensidade.png")


## Gráfico 3: Velocidade máxima

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x="Jogador", y="Velocidade maxima [km/h]", data=kpis, palette="Greens_d")
plt.xticks(rotation=90)
plt.title("Velocidade Máxima por Jogador")
plt.ylabel("Velocidade (km/h)")
plt.tight_layout()
plt.savefig("grafico_velocidade_maxima.png")


## Gráfico 4: Número de Sprints

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x="Jogador", y="Numero Sprints", data=kpis, palette="Reds")
plt.xticks(rotation=90)
plt.title("Número de Sprints por Jogador")
plt.ylabel("Sprints")
plt.tight_layout()
plt.savefig("grafico_sprints.png")


## Gráfico 5: Acelerações vs Desacelerações

In [ ]:
kpis_melt = kpis.melt(id_vars=["Jogador"], value_vars=["Total Acel", "Total Desacel"],
                      var_name="Tipo", value_name="Quantidade")

plt.figure(figsize=(10, 6))
sns.barplot(x="Jogador", y="Quantidade", hue="Tipo", data=kpis_melt)
plt.xticks(rotation=90)
plt.title("Acelerações e Desacelerações por Jogador")
plt.tight_layout()
plt.savefig("grafico_acel_desacel.png")


## Gráfico 6: Tempo em Zonas Cardéacas Altas

In [ ]:
cardio = kpis[["Jogador", "Min FC Z4", "Min FC Z5"]].melt(id_vars="Jogador", 
                                                          var_name="Zona FC", 
                                                          value_name="Minutos")

plt.figure(figsize=(10, 6))
sns.barplot(x="Jogador", y="Minutos", hue="Zona FC", data=cardio)
plt.xticks(rotation=90)
plt.title("Tempo nas Zonas de Frequência Cardíaca Alta")
plt.tight_layout()
plt.savefig("grafico_fc_altas.png")


## Gráfico 7: Comparação por posição (Ex: distânica)

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(x="Posicao do jogador", y="Dist Total [m]", data=kpis)
plt.title("Distância Total por Posição")
plt.tight_layout()
plt.savefig("boxplot_distancia_posicao.png")
